# Part 1) A linear programming approach for optimizing features in ML models

Read the article [A linear programming approach for optimizing features in ML models](https://engineering.fb.com/2021/07/29/data-infrastructure/linear-programming/).

Summarise it technically in about **400 words**, ensuring you capture the Mathematical formulation of how Linear Programming is used. (Do not discuss the code.)

- Use LaTeX for the mathematical formulae, not images. You also need to expand on the formulae given in the article.

## Article's summary
### **Technical Summary of the Article**  

The article discusses how **Facebook (Meta)** uses **Linear Programming (LP)** to optimize feature selection in **machine learning (ML) models** while managing **infrastructure constraints** (storage, CPU, memory). The key challenge is balancing **model performance gains** from new features against **limited computational resources**.  

#### **Mathematical Formulation**  
The problem is framed as a **binary optimization problem** where:  
- **Objective:** Maximize total **feature gain** (model performance improvement).  
- **Constraints:** Infrastructure limits (Service A and B capacities).  

**Variables:**  
- Let \( x_f \) = binary vector (1 if feature is selected uncompressed, 0 otherwise).  
- Let \( x_c \) = binary vector (1 if feature is selected compressed, 0 otherwise).  

**Objective Function:**  
$$
\text{Maximize} \quad G = g_f^T x_f + g_c^T x_c
$$  
where:  
- \( g_f \) = gain vector for uncompressed features.  
- \( g_c \) = gain vector for compressed features (reduced due to compression).  

**Constraints:**  
1. **Mutual Exclusivity:** A feature cannot be both compressed and uncompressed.  
   $$
   x_f + x_c \leq \mathbf{1}
   $$  
2. **Service A Capacity (e.g., Storage):**  
   $$
   s_{A_f}^T x_f + s_{A_c}^T x_c \leq C_A
   $$  
   where \( s_{A_f}, s_{A_c} \) = storage costs in Service A.  
3. **Service B Capacity (e.g., CPU/Memory for Compression):**  
   $$
   s_{B_f}^T x_f + s_{B_c}^T x_c \leq C_B
   $$  
4. **Compression Overhead:** Some features require Service B resources for compression.  

#### **Key Insights**  
- **Trade-offs:**  
  - A high-gain feature may consume too much storage, while multiple smaller features could yield better cumulative gain.  
  - Compression reduces storage but may decrease gain and require additional CPU/memory.  
- **Sensitivity Analysis:**  
  - The LP model helps quantify how much **adding capacity** (e.g., more storage) improves model performance.  
  - It reveals **interdependencies** (e.g., increasing Service B capacity reduces reliance on Service A).  

#### **Practical Applications**  
- **Automated Feature Selection:** Replaces manual decision-making with an **optimized, scalable solution**.  
- **Infrastructure Planning:** Identifies which services need upgrades for maximum ROI.  

This approach ensures **efficient resource allocation** while maintaining **model performance**, demonstrating how LP bridges **ML optimization** and **infrastructure management**.  

# Part 2) Farmer's Problem

A farmer has 500 acres of land to allocate to wheat, corn, and sugar beets.

The following table summarises the requirements and constraints:

|                          | Unit    | Wheat | Corn | Sugar Beets |
|--------------------------|---------|------:|-----:|:-----------:|
| Yield                    | T/acre  | 2.5   | 3    | 20          |
| Demand (Need for feed)   | T       | 200   | 240  |             |
| Planting cost            | £/acre  | 150   | 230  | 260         |
| Selling price            | £/T     | 170   | 150  | 36 if produce ≤ 6000 T |
|                          | £/T     |       |      | 10 if produce > 6000 T |
| Backup (Purchase price)  | £/T     | 238   | 210  |             |

## Mathematical formulation

|Variable name| Description |
|:------------|:-----|
|$x_1$| Acres of land used for wheat |
|$x_2$| Acres of land used for corn |
|$x_3$| Acres of land used for sugar beets |
|$p_1$| Tons of crop wheat sold |
|$p_2$| Tons of crop corn sold |
|$p_3$| Tons of crop sugar beets sold at £36 |
|$p_4$| Tons of crop sugar beets sold at £10 |
|$y_1$| Tons of wheat purchased |
|$y_2$| Tons of corn purchased |

### Mathematical Formulation

**Profit formula:**
$$
\text{Maximize } Z = (170p_1 + 150p_2 + 36p_3 + 10p_4) - (150x_1 + 230x_2 + 260x_3) - (238y_1 + 210y_2)
$$

**Constraints:**
$$
\begin{alignat*}{4}
    x_1 + x_2 + x_3 &\leq 500 & \quad &\text{(Land constraint)} \\
    2.5x_1 + y_1 - p_1 &\geq 200 & \quad &\text{(Wheat demand)} \\
    3x_2 + y_2 - p_2 &\geq 240 & \quad &\text{(Corn demand)} \\
    p_3 &\leq 6000 & \quad &\text{(Premium sugar beets limit)} \\
    20x_3 - p_3 - p_4 &= 0 & \quad &\text{(Sugar beets production)} \\
    x_1, x_2, x_3, p_1, p_2, p_3, p_4, y_1, y_2 &\geq 0 & \quad &\text{(Non-negativity)}
\end{alignat*}
$$



## Solution using PuLP

In [1]:
from pulp import *

In [1]:
from pulp import *

# Create the problem
prob = LpProblem("Farmer_Problem", LpMaximize)

# Decision variables
x1 = LpVariable("Wheat_acres", 0, None, LpInteger)
x2 = LpVariable("Corn_acres", 0, None, LpInteger)
x3 = LpVariable("SugarBeets_acres", 0, None, LpInteger)
p1 = LpVariable("Wheat_sold", 0, None, LpInteger)
p2 = LpVariable("Corn_sold", 0, None, LpInteger)
p3 = LpVariable("SugarBeets_sold_high", 0, 6000, LpInteger)
p4 = LpVariable("SugarBeets_sold_low", 0, None, LpInteger)
y1 = LpVariable("Wheat_purchased", 0, None, LpInteger)
y2 = LpVariable("Corn_purchased", 0, None, LpInteger)

# Objective function
prob += (170*p1 + 150*p2 + 36*p3 + 10*p4) - (150*x1 + 230*x2 + 260*x3) - (238*y1 + 210*y2)

# Constraints
prob += x1 + x2 + x3 <= 500
prob += 2.5*x1 + y1 - p1 >= 200
prob += 3*x2 + y2 - p2 >= 240
prob += 20*x3 - p3 - p4 == 0

# Solve the problem
prob.solve()

# Print results
print("Status:", LpStatus[prob.status])
print("Total Profit = £", value(prob.objective))

Status: Optimal
Total Profit = £ 118600.0


### Optimal solution

|Category |Unit|Wheat|Corn|Sugar Beets|
|---------|----|-----|----|-----------|
|Area     |Acre| 120 |80  |300        |
|Yield    |T   | 300 | 240|   6000    |
|Sales    |T   | 100 |  0 |      6000 |
|Purchase |T   |  0  | 0  |         - |

**Total cost: £118,600**

### Profit Calculation for Optimal Solution

#### Revenue:
1. **Wheat Sales**  
   `100 T × £170/T = £17,000`

2. **Corn Sales**  
   `0 T × £150/T = £0`

3. **Sugar Beets Sales**  
   `6,000 T × £36/T = £216,000`

**Total Revenue**  
`£17,000 + £0 + £216,000 = £233,000`

---

#### Costs:
1. **Planting Costs**  
   - Wheat: `120 acres × £150/acre = £18,000`  
   - Corn: `80 acres × £230/acre = £18,400`  
   - Sugar Beets: `300 acres × £260/acre = £78,000`  
   
   **Total Planting Costs**  
   `£18,000 + £18,400 + £78,000 = £114,400`

2. **Purchase Costs**  
   - Wheat Purchased: `0 T × £238/T = £0`  
   - Corn Purchased: `0 T × £210/T = £0`  
   
   **Total Purchase Costs**  
   `£0 + £0 = £0`

**Total Costs**  
`£114,400 + £0 = £114,400`

---

#### Net Profit:
`Profit = Total Revenue - Total Costs = £233,000 - £114,400 = £118,600`

### Verification Summary

| Component        | Calculation              | Amount (£) |
|------------------|--------------------------|-----------:|
| **Total Revenue** | 17000 + 0 + 216000       | 233,000    |
| **Total Costs**   | 18400 + 18000 + 78000 + 0 | 114,400    |
| **Net Profit**    | 233000 - 114400          | **118,600** |

All constraints are satisfied and the solution is optimal.